In [62]:
from time import sleep
import pandas as pd
import subprocess
import csv

mainPath = r"C:/HDI/lib"
import os, sys
bits="64bit" if sys.maxsize > 2**32 else "32bit"
# load the appropriate version of the library.  Change to point to local library location as appropriate.
libpath="C:/Python Scripts/watersimgreaderpy/" + bits
os.chdir(libpath)
import WatersIMGReader as wat 
from ctypes import *
import numpy as np
reader = cdll.LoadLibrary("C:/Python Scripts/watersimgreaderpy/64bit/watersimgreader.dll") # This will need to change 



In [142]:
rawdata = '2024_test_test_0012.raw'
datadir = 'C:/HDI/data/'
outdata = rawdata[0:-4]
outdir = datadir + 'outputs/'
csvoutdir = datadir + 'CSVoutputs/'

In [152]:
fulldata = datadir + rawdata
fulloutdata = outdir + outdata
fulloutCSVdata = csvoutdir + outdata

In [153]:
os.makedirs(outdir, exist_ok=True) 
os.makedirs(csvoutdir, exist_ok=True)

In [137]:
WellList=['A01','A02','A03','A04','A05','A06','B01','B02','B03','B04']
print(len(WellList))

10


In [70]:
def create_series_file(start, end, filename="C:/HDI/lib/outputScans.txt"):
    with open(filename, "w") as file:
        for i in range(start, end + 1):
            file.write(f"1\t{i}\n")

In [154]:
def create96_well_process(Rwell):
    prog = 'C:/HDI/lib/maldichrom.exe'
    raw_in = fulldata
    raw_out = f'{fulloutdata}_{Rwell}.raw'
    scans = 'C:/HDI/lib/outputScans.txt'
    command = f'{prog} -d "{raw_in}" -p "{scans}" -w "{raw_out}"'
    subprocess.Popen(command,cwd=mainPath,shell=True)

In [155]:
def raw_to_csv(Rwell):
    reader=wat.WatersIMGReader(f'{fulloutdata}_{Rwell}.raw',1)
    masses, intens, npoints = reader.getCombinedScans(1,1,0,0)
    with open(f'{fulloutCSVdata}_{Rwell}.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(len(masses)):
            writer.writerow([masses[i], intens[i]])
    

In [93]:
try:
    reader=wat.WatersIMGReader(fulldata,1)
    
    scans = reader.getTotalScans()
    print("Data contains: " + str(scans) + " scans")
    
    massRange = reader.getMassRange()
    print("Mass range: " + str(massRange[0]) + " to " + str(massRange[1]))
  
       
except NameError as err:
    print(err)

Data contains: 100 scans
Mass range: 50.0 to 1200.0


In [94]:
# get X,Y coordinates from the data
X,Y,points = reader.getXYCoordinates()
Ynp=np.array(Y)
ScanR=np.arange(scans)+1
Number_of_wells = len(np.unique(Ynp))
print(Number_of_wells)

len1, len2 = len(ScanR), len(Ynp)
if len2 < len1:
    padding = np.full(len1-len2,Ynp[-1])
    Ynp2=np.concatenate([Ynp,padding])

10


In [158]:
for x in range(Number_of_wells):
    Rwell = WellList[x]
    Data=np.column_stack((ScanR,Ynp2))
    Output=Data[np.where(Data[:,1]==x),0]
    start = int(Output[0,0])
    end = int(Output[0,-1])
    create_series_file(start, end)
    create96_well_process(Rwell)
    sleep(0.5)
    raw_to_csv(Rwell)
    # Read the Well ID (if available)
    # Write the new raw files
    # Write to the csv file - this will be read by the next app. 
    print(x)

0
1
2
3
4
5
6
7
8
9
